In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_17907/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_17907/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_6_22_7,0.700237,0.554157,0.648715,0.428009,0.046668,0.264672,0.246082,0.045699,0.091210,0.216027,0.628178,0.217708,56.129407,127.817768,"Hidden Size=[4, 2], regularizer=0.02, learning..."
1,model_6_22_3,0.702623,0.546214,0.637644,0.483292,0.046296,0.269387,0.253838,0.041282,0.090941,0.215166,0.631138,0.216840,56.145388,127.833749,"Hidden Size=[4, 2], regularizer=0.02, learning..."
2,model_1_36_2,0.677328,0.544478,0.557180,0.646694,0.050234,0.270418,0.310205,0.042272,0.093006,0.224130,0.628350,0.225874,39.982117,88.730202,"Hidden Size=[4], regularizer=0.02, learning_ra..."
3,model_1_35_7,0.676387,0.544416,0.556500,0.546486,0.050381,0.270454,0.310681,0.075250,0.093178,0.224456,0.627267,0.226203,39.976296,88.724382,"Hidden Size=[4], regularizer=0.02, learning_ra..."
4,model_1_36_6,0.678556,0.544319,0.557659,0.645837,0.050043,0.270512,0.309869,0.042375,0.092822,0.223703,0.629765,0.225444,39.989745,88.737831,"Hidden Size=[4], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,model_8_16_2,0.887447,-1.891187,-2.333653,0.793950,0.017522,1.716334,2.335293,0.030704,0.048408,0.132373,-0.814920,0.133403,250.088541,597.060210,"Hidden Size=[12, 6], regularizer=0.02, learnin..."
1454,model_8_16_1,0.871811,-1.911068,-2.360395,0.835209,0.019957,1.728136,2.354027,0.024556,0.050720,0.141268,-1.067046,0.142368,249.828382,596.800051,"Hidden Size=[12, 6], regularizer=0.02, learnin..."
1455,model_7_26_8,0.837877,-2.117597,-0.201791,0.335750,0.025240,1.850740,0.841880,0.058175,0.061563,0.158870,0.673220,0.160106,137.358677,323.748416,"Hidden Size=[8, 4], regularizer=0.02, learning..."
1456,model_7_26_2,0.677638,-2.408639,-0.715978,0.317564,0.050186,2.023515,1.202078,0.059768,0.084930,0.224022,0.350240,0.225765,135.984044,322.373783,"Hidden Size=[8, 4], regularizer=0.02, learning..."
